In [6]:
import numpy as np
import pandas as pd
from geopy.distance import distance
import matplotlib.pyplot as plt
%matplotlib inline
import warnings
warnings.filterwarnings("ignore")
import matplotlib.dates as mdates
import matplotlib.ticker as plticker
from scipy import integrate
from math import ceil
import math
import urllib.request
import requests
import json
import sqlalchemy
from sqlalchemy import create_engine
import random

In [7]:
#connecting to database server
engine = sqlalchemy.create_engine('sqlite:///mdb.db') 


In [1]:
key_routes = ["B42","B45","B47","B51","B52","B59","B60","B61","B65","B73","B80","B84","B86","B94","B95"]
avg_rider = [30,35,25,35,20,20,35,50,30,60,20,60,20,50,70]

In [4]:
#storing ridership information in dataframe 
ridership = pd.DataFrame (list(zip(key_routes, avg_rider)), 
               columns =['key_routes', 'avg_rider']) 

In [9]:
query = "SELECT * FROM `comune di milano_stop_times` RIGHT JOIN `comune di milano_stops` ON `comune di milano_stops`.stop_id = `comune di milano_stop_times`.stop_id LEFT JOIN `comune di milano_trips` ON `comune di milano_stop_times`.trip_id = `comune di milano_trips`.trip_id;"
data = pd.read_sql(query, engine).drop(columns = "index")
data = data.loc[:,~data.columns.duplicated()]
data.head()

,trip_id,arrival_time,departure_time,stop_id,stop_sequence,stop_name,stop_lat,stop_lon,zone_id,parent_station,route_id,service_id,direction_id
0,20200314_8_FESTIVO_CIMITERI_fes_i_171-172-174-...,16:47:00,16:47:00,15278,1,ingresso,45.529103,9.181773,None,None,B176,20200314_8_FESTIVO_CIMITERI_fes_i_171-172-174-176,0.0
1,20200314_8_FESTIVO_CIMITERI_fes_i_171-172-174-...,16:32:00,16:32:00,15278,1,ingresso,45.529103,9.181773,None,None,B176,20200314_8_FESTIVO_CIMITERI_fes_i_171-172-174-176,0.0
2,20200314_8_FESTIVO_CIMITERI_fes_i_171-172-174-...,16:17:00,16:17:00,15278,1,ingresso,45.529103,9.181773,None,None,B176,20200314_8_FESTIVO_CIMITERI_fes_i_171-172-174-176,0.0
3,20200314_8_FESTIVO_CIMITERI_fes_i_171-172-174-...,16:02:00,16:02:00,15278,1,ingresso,45.529103,9.181773,None,None,B176,20200314_8_FESTIVO_CIMITERI_fes_i_171-172-174-176,0.0
4,20200314_8_FESTIVO_CIMITERI_fes_i_171-172-174-...,15:47:00,15:47:00,15278,1,ingresso,45.529103,9.181773,None,None,B176,20200314_8_FESTIVO_CIMITERI_fes_i_171-172-174-176,0.0


In [11]:
#taking only key bus routes for boston 
df = data[data["route_id"].isin(key_routes)]

#removing duplicate columns (stop_id ) due to joining 
df = df.loc[:,~df.columns.duplicated()]
df.head()

,trip_id,arrival_time,departure_time,stop_id,stop_sequence,stop_name,stop_lat,stop_lon,zone_id,parent_station,route_id,service_id,direction_id
19371,20200314_8_GIORNALIERO (FESTIVO)_no rete not_f...,23:35:00,23:35:00,10631,2,p.za ghirlandaio,45.466941,9.146917,None,None,B80,20200314_8_GIORNALIERO (FESTIVO)_no rete not_f...,1.0
19372,20200314_8_GIORNALIERO (FESTIVO)_no rete not_f...,20:04:00,20:04:00,10631,2,p.za ghirlandaio,45.466941,9.146917,None,None,B80,20200314_8_GIORNALIERO (FESTIVO)_no rete not_f...,1.0
19379,20200314_8_GIORNALIERO (FESTIVO)_no rete not_f...,20:55:24,20:55:24,12434,5,l.go rio de janeiro,45.473468,9.224988,None,None,B61,20200314_8_GIORNALIERO (FESTIVO)_no rete not_f...,0.0
19380,20200314_8_GIORNALIERO (FESTIVO)_no rete not_f...,06:50:24,06:50:24,12434,5,l.go rio de janeiro,45.473468,9.224988,None,None,B61,20200314_8_GIORNALIERO (FESTIVO)_no rete not_f...,0.0
19403,20200314_8_GIORNALIERO (FESTIVO)_no rete not_f...,23:17:50,23:17:50,12962,4,via marochetti via avezzana,45.438883,9.226796,None,None,B95,20200314_8_GIORNALIERO (FESTIVO)_no rete not_f...,0.0


In [12]:
#grouping data based on route id and stop sequence 
grp = df.groupby(["route_id", "stop_sequence"])
grp.first()

trip_id  \
route_id stop_sequence                                                      
B42      1              20200314_8_GIORNALIERO (FESTIVO)_no rete not_f...   
         2              20200314_8_GIORNALIERO (FESTIVO)_no rete not_f...   
         3              20200314_8_GIORNALIERO (FESTIVO)_no rete not_f...   
         4              20200314_8_GIORNALIERO (FESTIVO)_no rete not_f...   
         5              20200314_8_GIORNALIERO (FESTIVO)_no rete not_f...   
...                                                                   ...   
B95      26             20200314_8_GIORNALIERO (FESTIVO)_no rete not_f...   
         27             20200314_8_GIORNALIERO (FESTIVO)_no rete not_f...   
         28             20200314_8_GIORNALIERO (FESTIVO)_no rete not_f...   
         29             20200314_8_GIORNALIERO (FESTIVO)_no rete not_f...   
         30             20200314_8_GIORNALIERO (FESTIVO)_no rete not_f...   

                       arrival_time departure_time stop_id  \
route_id stop_sequence                                       
B42      1                 15:30:00       15:30:00   11921   
         2                 17:12:42       17:12:42   12010   
         3                 17:13:24       17:13:24   12317   
         4                 17:14:23       17:14:23   12305   
         5                 17:15:15       17:15:15   12303   
...                             ...            ...     ...   
B95      26                23:42:54       23:42:54   17460   
         27                23:43:48       23:43:48   17461   
         28                23:45:23       23:45:23   10516   
         29                23:46:13       23:46:13   10513   
         30                23:47:00       23:47:00   10510   

                                        stop_name   stop_lat  stop_lon  \
route_id stop_sequence                                                   
B42      1                stazione centrale m2 m3  45.486831  9.202492   
         2                       p.za s. giuseppe  45.526123  9.205752   
         3                via arezzo v.le suzzani  45.525476  9.208008   
         4                v.le suzzani via ponale  45.523514  9.207330   
         5              v.le suzzani v.le berbera  45.521554  9.206206   
...                                           ...        ...       ...   
B95      26                via tobagi via olgiati  45.440073  9.146070   
         27                via tobagi v.le faenza  45.439245  9.143252   
         28                          via giussani  45.440948  9.139863   
         29               via pepere via giussani  45.439861  9.138015   
         30                           q.re barona  45.438122  9.138977   

                       zone_id parent_station  \
route_id stop_sequence                          
B42      1                None           None   
         2                None           None   
         3                None           None   
         4                None           None   
         5                None           None   
...                        ...            ...   
B95      26               None           None   
         27               None           None   
         28               None           None   
         29               None           None   
         30               None           None   

                                                               service_id  \
route_id stop_sequence                                                      
B42      1              20200314_8_GIORNALIERO (FESTIVO)_no rete not_f...   
         2              20200314_8_GIORNALIERO (FESTIVO)_no rete not_f...   
         3              20200314_8_GIORNALIERO (FESTIVO)_no rete not_f...   
         4              20200314_8_GIORNALIERO (FESTIVO)_no rete not_f...   
         5              20200314_8_GIORNALIERO (FESTIVO)_no rete not_f...   
...                                                                   ...   
B95      26             20200314_8_GIORN

In [13]:
#Counting number of trips at each stop and evaluating the origin and destination of trips  

def trip_agg(key_routes):
    trip_list = []
    count = []
    routes = []
    for route in key_routes: 
        p = list(grp["stop_sequence"].value_counts().loc[route])
        q = []
        for i in range(0,len(p)-1):
            q.append(p[i] - p[i+1])
        q.append(p[-1])
        res = []
        for idx, val in enumerate(q):
            if val > 0:
                res.append (float(idx+1))
                count.append(abs(val))
            elif val < 0:
                res.append(float(idx+2))
                count.append(abs(val))
        a = list(grp.get_group((route,res[::-1][0])).trip_id)  
        trip_list.append(random.choice(a))
        routes.append(route)
        #generating a random trip from a set of unique trips between two given stops 
        for i in range(0,len(res)-1): 
            a = list(grp.get_group((route,res[::-1][i])).trip_id)  
            b = list(grp.get_group((route,res[::-1][i+1])).trip_id)
            routes.append(route)
            if set(a) == set(b):
                a = list(grp.get_group((route,res[::-1][i]+1)).trip_id)  
                trip_list.append(random.choice(list(set(b)-set(a))))
            elif set(a) > set(b): 
                trip_list.append(random.choice(list(set(a)-set(b))))
            else :
                trip_list.append(random.choice(list(set(b)-set(a))))
    return count, trip_list, routes 


In [14]:
count, trip_list, route_id = trip_agg(key_routes)

In [16]:
trip_data = pd.DataFrame(list(zip(route_id, trip_list,count)),  columns = ["route_id", "trip_id","trip_count"])
#storing estiamted data in database 
trip_data.to_sql('cdm_route_data', con=engine, if_exists = 'replace')

In [17]:
#estimating mass of the bus 

p_wt = 70 #70 kg is passenger weight 

#take bus type information from the database 
query = "SELECT * FROM bus_library ;"
bus_data = pd.read_sql(query, engine)

#load the route data 
query = "SELECT * FROM cdm_route_data ;"
trip_data = pd.read_sql(query, engine)

# #calculate the ridership 
for j in bus_data.index: 
    bus = bus_data.iloc[j]["index"]+ "_wt"
    trip_data[bus] = 0
    for i in ridership.index:
        trip_data[bus][trip_data["route_id"] == ridership.iloc[i]["key_routes"]] = bus_data.iloc[j]["empt_wt"] + p_wt*ridership.iloc[i]["avg_rider"]

ProgrammingError: (mysql.connector.errors.ProgrammingError) 1146 (42S02): Table 'mdb.bus_library' doesn't exist
[SQL: SELECT * FROM bus_library ;]
(Background on this error at: http://sqlalche.me/e/f405)